In [403]:
import sqlite3
import pandas as pd
import os
import datetime as datetime
import numpy as np
from dotenv import dotenv_values
from binance.client import Client
import requests
import random
import yfinance as yf
import sys 
import math

In [404]:
is_auto=False

no_last_items=1
no_set=5
x_now=datetime.datetime.now()

# api_source='yahoo'
api_source='bn'

In [405]:
if is_auto==True:
    if len(sys.argv)==2:
       print(sys.argv[1] ) 
       api_source=sys.argv[1] 
        
       with open('readme.txt', 'w') as f:
        for i in range(len(sys.argv)):
         f.write(sys.argv[i]+"\n") 
    else:
        raise Exception("take api_source as param bn or yahoo")


#exit()


In [406]:
buildtime = x_now.strftime('%d%b%y_%H%M')
print(buildtime)

14May22_0448


In [407]:
# config_values= dotenv_values(dotenv_path=f'{path_config}\\.env')
config_values= dotenv_values(dotenv_path='.env')
api_key = config_values['bn_key']
api_secret =config_values['bn_secrete']
client=Client(api_key,api_secret)

line_token = config_values['line_token']

In [408]:
data_base_file=f'alert_price_db.sqlite3'
print("Connect SQLite")
conn = sqlite3.connect(os.path.abspath(data_base_file))

sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

Connect SQLite


In [409]:
if api_source=='bn':
    sql_price_range="select * from symbol_price_info where is_active=1 and api_source='bn'"

elif api_source=='yahoo':
    sql_price_range="select * from symbol_price_info where is_active=1 and api_source='yahoo'"
    
df=pd.read_sql_query(sql_price_range, conn)
df['symbol_pair']=df.apply( lambda x: f"{x.symbol_x}{x.symbol_pair}",axis=1)
df

#close conection from selecting fist
# conn.close()

,symbol_key,symbol_x,symbol_pair,price_range,is_active,api_source
0,eth,ETH,ETHBUSD,"1800,2100,2400,2650,2800",1,bn
1,btc,BTC,BTCBUSD,"27000,31000,34500,37500,40000",1,bn
2,bnb,BNB,BNBBUSD,"200,250,300,350,400",1,bn
3,st-busd,BUSD,BUSDUSDT,"0.9,0.925,0.95,0.975,1.1",1,bn
4,st-usdc,USDC,USDCUSDT,"0.9,0.925,0.95,0.975,1.1",1,bn


In [410]:
# get price from binace

# get price from yahoo
# https://aroussi.com/post/python-yahoo-finance
# https://github.com/ranaroussi/yfinance

current_price_dict={}
list_symbol=[]

if api_source=='bn':
    for index,item in df.iterrows():
        # current_price =(client.get_symbol_ticker(symbol=item["symbol_pair"]))['price']
        current_price=random.randint(1, 30)
        current_price_dict[item["symbol_key"]]= round( float(current_price),4)
        list_symbol.append(item["symbol_key"])
          
        
#auto_adjust = True
elif api_source=='yahoo':
    fund_symbols=df['symbol_x'].tolist()
    fund_keys=df['symbol_key'].tolist()
    
    symbol_str=' '.join(fund_symbols )
    list_current_price = yf.download( tickers = symbol_str,threads = True,
                                 group_by=True,period = "1d",prepost = True)
    list_fund= list(zip(fund_keys,fund_symbols))
    for x in list_fund:
     current_price_dict[x[0]]=round( list_current_price[x[1]]['Close'][0],2)
     list_symbol.append(x[0])

print(current_price_dict )  


{'eth': 13.0, 'btc': 24.0, 'bnb': 10.0, 'st-busd': 18.0, 'st-usdc': 1.0}


In [411]:
def get_last_record():
    conn = sqlite3.connect(os.path.abspath(data_base_file))

    df_all_last_update=pd.DataFrame()

    for item in list_symbol:
        sql=f"select * from notification_transaction where symbol_key='{item}' order by datetime desc limit 1"
        df_item=pd.read_sql_query(sql, conn)
        df_all_last_update=pd.concat([df_all_last_update,df_item])
    #     print(df_item)
    return df_all_last_update


In [412]:
df_all_prev=get_last_record()
df_all_prev

,datetime,symbol_key,price,all_ranges,no_ranges,idx_pos,idx_pct,msg,prev_msg,prev_pct_chg,prev_price
0,2022-05-14 04:47:45,eth,5.0,"1800,2100,2400,2650,2800",6,1,16.7,eth:5.0 < 1800.0,None,None,None
0,2022-05-14 04:47:45,btc,1.0,"27000,31000,34500,37500,40000",6,1,16.7,btc:1.0 < 27000.0,None,None,None
0,2022-05-14 04:47:45,bnb,3.0,"200,250,300,350,400",6,1,16.7,bnb:3.0 < 200.0,None,None,None
0,2022-05-14 04:47:45,st-busd,1.0,"0.9,0.925,0.95,0.975,1.1",6,5,83.3,0.975 <= st-busd:1.0 >= 1.1,None,None,None
0,2022-05-14 04:47:45,st-usdc,12.0,"0.9,0.925,0.95,0.975,1.1",6,6,100.0,st-usdc:12.0 > 1.1,None,None,None


In [413]:
dict_prev_item={}
for  symb in  list_symbol:
    df_temp=df_all_prev.query("symbol_key==@symb")
    str_prev_msg=None
    prev_price=None
    # print(df_temp)
    if  df_temp.empty==False:
      temp=df_temp.loc[0]
      str_prev_msg=f"{temp['idx_pos']}# {temp['msg']}"
      prev_price=  temp['price']

    dict_prev_item[symb]=(str_prev_msg,prev_price)
print(dict_prev_item)

{'eth': ('1# eth:5.0 < 1800.0', 5.0), 'btc': ('1# btc:1.0 < 27000.0', 1.0), 'bnb': ('1# bnb:3.0 < 200.0', 3.0), 'st-busd': ('5# 0.975 <= st-busd:1.0 >= 1.1', 1.0), 'st-usdc': ('6# st-usdc:12.0 > 1.1', 12.0)}


In [414]:
def find_range(item):
   symbol_key=item['symbol_key'] 

   p=current_price_dict[symbol_key]
   print(f'{symbol_key} : {p}') 

   priceList_str= item["price_range"].strip().split(",")
   priceList=[ float(x) for x in priceList_str]
   list.sort(priceList) 
#    print(type(priceList))
#    print(priceList)
     
   log_info={}

   log_info['datetime']=(x_now).strftime("%Y-%m-%d %H:%M:%S")
   log_info['symbol_key']=symbol_key   
   log_info['price']=p
   log_info['all_ranges']= item["price_range"]
   
   range_length=len(priceList)
   log_info['no_ranges']=range_length+1
   
   position=0;
   for index in range(0,range_length):   
#      print("iterate:",index)
     if (index==0) and (p<priceList[index]):
            position= 1
            log_info['msg']=f"{symbol_key}:{p} < {priceList[index]}"
            
     elif (index>0) and (index< range_length ) and ( p>=priceList[index-1] and p<=priceList[index] ):
            position=  index+1
            log_info['msg']= f"{priceList[index-1]} <= {symbol_key}:{p} >= {priceList[index] }"
          
     elif   (index==range_length-1) and (p>priceList[index])    :
            position= range_length+1
            log_info['msg']= f"{symbol_key}:{p} > {priceList[index]}"
            
            
   if  symbol_key in dict_prev_item:

        msg,prev_p=dict_prev_item[symbol_key]

        if msg is not None and prev_p is not None :
         log_info['prev_msg']=msg   
         log_info['prev_price']= prev_p
         log_info['prev_pct_chg']= round( (p-prev_p)/prev_p*100 ,1)
        else:
         log_info['prev_msg']=None   
         log_info['prev_price']=None   
         log_info['prev_pct_chg']=None
         
    
   log_info['idx_pos']=position 
   log_info['idx_pct']= round( log_info['idx_pos']/log_info['no_ranges']*100,1)  
                
#      else:
#         raise Exception("There are someting wrong!!!!")
        
   print(log_info)
   print("===========================================================")
   return log_info 
   
df['transaction']=df.apply(find_range,axis=1)  

eth : 13.0
{'datetime': '2022-05-14 04:48:20', 'symbol_key': 'eth', 'price': 13.0, 'all_ranges': '1800,2100,2400,2650,2800', 'no_ranges': 6, 'msg': 'eth:13.0 < 1800.0', 'prev_msg': '1# eth:5.0 < 1800.0', 'prev_price': 5.0, 'prev_pct_chg': 160.0, 'idx_pos': 1, 'idx_pct': 16.7}
btc : 24.0
{'datetime': '2022-05-14 04:48:20', 'symbol_key': 'btc', 'price': 24.0, 'all_ranges': '27000,31000,34500,37500,40000', 'no_ranges': 6, 'msg': 'btc:24.0 < 27000.0', 'prev_msg': '1# btc:1.0 < 27000.0', 'prev_price': 1.0, 'prev_pct_chg': 2300.0, 'idx_pos': 1, 'idx_pct': 16.7}
bnb : 10.0
{'datetime': '2022-05-14 04:48:20', 'symbol_key': 'bnb', 'price': 10.0, 'all_ranges': '200,250,300,350,400', 'no_ranges': 6, 'msg': 'bnb:10.0 < 200.0', 'prev_msg': '1# bnb:3.0 < 200.0', 'prev_price': 3.0, 'prev_pct_chg': 233.3, 'idx_pos': 1, 'idx_pct': 16.7}
st-busd : 18.0
{'datetime': '2022-05-14 04:48:20', 'symbol_key': 'st-busd', 'price': 18.0, 'all_ranges': '0.9,0.925,0.95,0.975,1.1', 'no_ranges': 6, 'msg': 'st-busd:18.

In [415]:
list_trans=[]
for index,item in df.iterrows():
    list_trans.append(item['transaction'])
df_tran=pd.DataFrame(list_trans)

df_tran['idx_pos']=df_tran['idx_pos'].astype('int')


print(df_tran.info())
df_tran

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datetime      5 non-null      object 
 1   symbol_key    5 non-null      object 
 2   price         5 non-null      float64
 3   all_ranges    5 non-null      object 
 4   no_ranges     5 non-null      int64  
 5   msg           5 non-null      object 
 6   prev_msg      5 non-null      object 
 7   prev_price    5 non-null      float64
 8   prev_pct_chg  5 non-null      float64
 9   idx_pos       5 non-null      int32  
 10  idx_pct       5 non-null      float64
dtypes: float64(4), int32(1), int64(1), object(5)
memory usage: 548.0+ bytes
None


,datetime,symbol_key,price,all_ranges,no_ranges,msg,prev_msg,prev_price,prev_pct_chg,idx_pos,idx_pct
0,2022-05-14 04:48:20,eth,13.0,"1800,2100,2400,2650,2800",6,eth:13.0 < 1800.0,1# eth:5.0 < 1800.0,5.0,160.0,1,16.7
1,2022-05-14 04:48:20,btc,24.0,"27000,31000,34500,37500,40000",6,btc:24.0 < 27000.0,1# btc:1.0 < 27000.0,1.0,2300.0,1,16.7
2,2022-05-14 04:48:20,bnb,10.0,"200,250,300,350,400",6,bnb:10.0 < 200.0,1# bnb:3.0 < 200.0,3.0,233.3,1,16.7
3,2022-05-14 04:48:20,st-busd,18.0,"0.9,0.925,0.95,0.975,1.1",6,st-busd:18.0 > 1.1,5# 0.975 <= st-busd:1.0 >= 1.1,1.0,1700.0,6,100.0
4,2022-05-14 04:48:20,st-usdc,1.0,"0.9,0.925,0.95,0.975,1.1",6,0.975 <= st-usdc:1.0 >= 1.1,6# st-usdc:12.0 > 1.1,12.0,-91.7,5,83.3


In [416]:
def insertMultipleItems(recordList):
    try:
        sqliteConnection = sqlite3.connect(os.path.abspath(data_base_file))
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        sqlite_insert_query = """
        INSERT INTO notification_transaction
        (datetime, symbol_key,price,all_ranges,no_ranges,
         msg,prev_msg,prev_price,prev_pct_chg,idx_pos,idx_pct) 
        VALUES (?,?,?, ?, ?,?,?,?,?,?,?);
         """

        cursor.executemany(sqlite_insert_query, recordList)
        sqliteConnection.commit()
        print("Total", cursor.rowcount, "Add transaction successfully")
        sqliteConnection.commit()
        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert multiple records into product table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("The SQLite connection is closed")


recordsToInsert=list(df_tran.to_records(index=False))
insertMultipleItems(recordsToInsert)

Connected to SQLite
Total 5 Add transaction successfully
The SQLite connection is closed


In [417]:
df_all_last_update=get_last_record()

df_all_last_update=df_all_last_update[['datetime','symbol_key','price','prev_price','prev_pct_chg','msg','prev_msg','idx_pct','idx_pos','all_ranges']]
df_all_last_update.fillna("-",inplace=True)
df_all_last_update.info()
df_all_last_update

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 0
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datetime      5 non-null      object 
 1   symbol_key    5 non-null      object 
 2   price         5 non-null      float64
 3   prev_price    5 non-null      float64
 4   prev_pct_chg  5 non-null      float64
 5   msg           5 non-null      object 
 6   prev_msg      5 non-null      object 
 7   idx_pct       5 non-null      float64
 8   idx_pos       5 non-null      int64  
 9   all_ranges    5 non-null      object 
dtypes: float64(4), int64(1), object(5)
memory usage: 440.0+ bytes


,datetime,symbol_key,price,prev_price,prev_pct_chg,msg,prev_msg,idx_pct,idx_pos,all_ranges
0,2022-05-14 04:48:20,eth,13.0,5.0,160.0,eth:13.0 < 1800.0,1# eth:5.0 < 1800.0,16.7,1,"1800,2100,2400,2650,2800"
0,2022-05-14 04:48:20,btc,24.0,1.0,2300.0,btc:24.0 < 27000.0,1# btc:1.0 < 27000.0,16.7,1,"27000,31000,34500,37500,40000"
0,2022-05-14 04:48:20,bnb,10.0,3.0,233.3,bnb:10.0 < 200.0,1# bnb:3.0 < 200.0,16.7,1,"200,250,300,350,400"
0,2022-05-14 04:48:20,st-busd,18.0,1.0,1700.0,st-busd:18.0 > 1.1,5# 0.975 <= st-busd:1.0 >= 1.1,100.0,6,"0.9,0.925,0.95,0.975,1.1"
0,2022-05-14 04:48:20,st-usdc,1.0,12.0,-91.7,0.975 <= st-usdc:1.0 >= 1.1,6# st-usdc:12.0 > 1.1,83.3,5,"0.9,0.925,0.95,0.975,1.1"


In [279]:
#!/usr/local/bin/python3
import requests
url = 'https://notify-api.line.me/api/notify'
token = line_token
headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer '+token}

In [280]:
no_split=math.ceil(len(df_all_last_update)/no_set)
print(no_split)
info_array = np.array_split(df_all_last_update,no_split)
for info in info_array:
    string_msg=info.to_string(index=False,header=True)
    string_msg='\n'+string_msg
    print(len(string_msg))
    r = requests.post(url, headers=headers, data = {'message':string_msg})
    print (r.text)
    print("===========================================") 

1
804
{"status":200,"message":"ok"}
